Following examples from Databricks Documentations:  
https://docs.databricks.com/applications/mlflow/tracking-examples.html#train-a-pyspark-model-and-save-in-mleap-format  
https://docs.databricks.com/applications/mlflow/model-examples.html#mleap-model-deployment-on-sagemaker

### Summary  
This notebook demonstrates the process of:
  - Training a **PySpark Pipeline** model (including cross-validation)
  - Logging/ Saving the trained model (the best model after hyper-parameter tuning) in **MLeap** format using **Mlflow**

#### Setup 
Launch a Python3 cluster with Databricks Runtime 6.0 with:
 - instance type: m4.large
 - spark config: `spark.databricks.mlflow.trackMLlib.enabled true` # helps to automatically track parameters when using `pyspark.ml.tuning.CrossValidator()`
 - an AWS IAM role: with full access to AWS SageMaker
 - libraries:
   - PyPI:
     - `boto3==1.9.215`
     - `mleap==0.8.1`
     - `mlflow[extra]`
     - `sagemaker==1.42.4`
   - Maven:
     - `ml.combust.mleap:mleap-spark_2.11:0.13.0`
     - `ml.dmlc:xgboost4j-spark:0.90`

In [4]:
%sh
lsb_release -a
python --version
mlflow --version
java -version

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial
Python 3.7.3
mlflow, version 1.3.0
openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~16.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)

#### Training (including CV)

This model uses the [20 Newsgroups dataset](http://kdd.ics.uci.edu/databases/20newsgroups/20newsgroups.html) which consists of articles from 20 Usenet newsgroups.  
This dataset is **pre-loaded** into Databricks in parquet format under `file:/dbfs/databricks-datasets/news20.binary/data-001/training`

In [6]:
# Load training data
df = spark.read.parquet("/databricks-datasets/news20.binary/data-001/training").select("text", "topic")
train, test = df.randomSplit([0.8, 0.2], 42)
train.cache().count()
test.cache().count()
display(train)

text topic cs.utexas.edu!geraldo.cc.utexas.edu!portal.austin.ibm.com!awdprime.austin.ibm.com!karner Subject: Re: Islamic marriage? From: karner@austin.ibm.com (F. Karner) <1993Apr2.103237.4627@Cadence.COM> Organization: IBM Advanced Workstation Division Originator: frank@karner.austin.ibm.com Lines: 50 In article <1993Apr2.103237.4627@Cadence.COM>, mas@Cadence.COM (Masud Khan) writes: > In article karner@austin.ibm.com (F. Karner) writes: > > > >Okay. So you want me to name names? There are obviously no official > >records of these pseudo-marriages because they are performed for > >convenience. What happens typically is that the woman is willing to move > >in with her lover without any scruples or legal contracts to speak of. > >The man is merely utilizing a loophole by entering into a temporary > >religious "marriage" contract in order to have sex. Nobody complains, > >nobody cares, nobody needs to know. > > > >Perhaps you should alert your imam. It could be that this practice is > >far more widespread than you may think. Or maybe it takes 4 muslim men > >to witness the penetration to decide if the practice exists! > >-- > > > > Again you astound me with the level of ignorance you display, Muslims > are NOT allowed to enter temporary marriages, got that? There is > no evidence for it it an outlawed practise so get your facts > straight buddy. Give me references for it or just tell everyone you > were lying. It is not a widespread as you may think (fantasise) in > fact contrary to your fantasies it is not practised at all amongst > Muslims. First of all, I'm not your buddy! Second, read what I wrote. I'm not talking about what muslims are ALLOWED to do, merely what *SOME* practice. They consider themselves as muslim as you, so don't retort with the old and tired "they MUST NOT BE TRUE MUSLIMS" bullshit. If I gave you the names what will you do with this information? Is a fatwa going to be leashed out against the perpetrators? Do you honestly think that someone who did it would voluntarily come forward and confess? With the kind of extremism shown by your co-religionaries? Fat chance. At any rate, there can be no conclusive "proof" by the very nature of the act. Perhaps people that indulge in this practice agree with you in theory, but hope that Allah will forgive them in the end. I think it's rather arrogant of you to pretend to speak for all muslims in this regard. Also, kind of silly. Are you insinuating that because the Koranic law forbids it, there are no criminals in muslim countries? This is as far as I care to go on this subject. The weakness of your arguments are for all netters to see. Over and out... -- DISCLAIMER: The opinions expressed in this posting are mine solely and do not represent my employer in any way. F. A. Karner AIX Technical Support | karner@austin.vnet.ibm.com alt.atheism egsner!ernest!m2.dseg.ti.com!tilde.csc.ti.com!mksol!kerr.dseg.ti.com!kkerr@mkcase1.dseg.ti.com From: kkerr@mkcase1.dseg.ti.com@MK (Kevin Kerr) Subject: Re: YANKKES 1 GAME CLOSER Organization: ENGINEERING AUTOMATION Lines: 38 Nntp-Posting-Host: kerr.dseg.ti.com In article <1993Apr6.233805.29755@freenet.carleton.ca> aa649@Freenet.carleton.ca (Ralph Timmerman) writes: >From: aa649@Freenet.carleton.ca (Ralph Timmerman) >Subject: Re: YANKKES 1 GAME CLOSER >Date: Tue, 6 Apr 1993 23:38:05 GMT >In a previous article, 002251w@axe.acadiau.ca (JASON WALTER WORKS) says: >> The N.Y.Yankees, are now one game closer to the A.L.East pennant. They >>clobbered Cleveland, 9-1, on a fine pitching performance by Key, and two >>homeruns by Tartabull(first M.L.baseball to go out this season), and a three >>run homer by Nokes. For all of you who didn't pick Boggs in your pools, >>tough break, he had a couple hits, and drove in a couple runs(with many more >>to follow). The Yanks beat an up and coming team of youngsters in the >>Indians. The Yankees only need to win 95 more games to get the division. >> GO YANKS., Mattingly for g.glove, and MVP, and Abbot fo

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, Tokenizer, HashingTF
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Define pipeline components
labelIndexer = StringIndexer(inputCol="topic", outputCol="label", handleInvalid="keep")
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="features")
dt = DecisionTreeClassifier()

# Construct a Pipeline object using the defined components
pipeline = Pipeline(stages=[labelIndexer, tokenizer, hashingTF, dt])

# Fine-tune the model, for simplicity purposes, only tune the 'numFeatures' parameter in HashingTF() vectorizer
paramGrid = (ParamGridBuilder()
             .addGrid(hashingTF.numFeatures, [100, 200])
             .build()
            )
cv = CrossValidator(estimator=pipeline, 
                    evaluator=MulticlassClassificationEvaluator(), 
                    estimatorParamMaps=paramGrid
                   )

In [8]:
# use Mlflow to log the best model into Mleap format
import mlflow
import mlflow.mleap
with mlflow.start_run() as run:
  cvmodel = cv.fit(train)
  # extract the optimal Pipeline model to be logged
  model = cvmodel.bestModel
  mlflow.mleap.log_model(spark_model=model, sample_input=train, artifact_path="test_model_dbr60")

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

#### Deploy to AWS SageMaker

##### Before deploy remotely to SageMaker, run a local test   
  - setup proper AWS authorization and Databricks CLI
  - pip install mlflow==1.3.0 locally 
  - install docker locally and start docker service
  - locally build a docker image and push it to AWS ECR:
    - `$ mlflow sagemaker build-and-push-container`
  - use Databricks CLI to copy the logged model to a local directory:
    - `$ databricks fs cp -r </databricks/path/to/logged/test_model/dbr60> </local/path/to/test_model_dbr60/>`
  - run local test:
    - `$ mlflow sagemaker run-local -m </local/path/to/test_model_dbr60/>`

##### ERROR MESSAGE:  
```
Using the mleap flavor for local serving!
2019/10/04 18:53:23 INFO mlflow.sagemaker: launching docker image with path /Users/a_frustrated_user/Downloads/test_model_dbr60
2019/10/04 18:53:23 INFO mlflow.sagemaker: executing: docker run -v /Users/a_frustrated_user/Downloads/test_model_dbr60:/opt/ml/model/ -p 5000:8080 -e MLFLOW_DEPLOYMENT_FLAVOR_NAME=mleap --rm mlflow-pyfunc serve
Error: A JNI error has occurred, please check your installation and try again
Exception in thread "main" java.lang.NoClassDefFoundError: org/eclipse/jetty/util/thread/ThreadPool
	at java.lang.Class.getDeclaredMethods0(Native Method)
	at java.lang.Class.privateGetDeclaredMethods(Class.java:2701)
	at java.lang.Class.privateGetMethodRecursive(Class.java:3048)
	at java.lang.Class.getMethod0(Class.java:3018)
	at java.lang.Class.getMethod(Class.java:1784)
	at sun.launcher.LauncherHelper.validateMainClass(LauncherHelper.java:544)
	at sun.launcher.LauncherHelper.checkAndLoadMain(LauncherHelper.java:526)
Caused by: java.lang.ClassNotFoundException: org.eclipse.jetty.util.thread.ThreadPool
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:349)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	... 7 more
Got sigterm signal, exiting.
```

`$ mlflow --version`  
  mlflow, version 1.3.0  
`$ aws --version`  
  aws-cli/1.16.225 Python/3.7.3 Darwin/18.7.0 botocore/1.12.215  
`$ databricks --version`  
  Version 0.8.7  
`$ docker version`  
  Client: Docker Engine - Community  
   Version:           19.03.2  
   API version:       1.40  
   Go version:        go1.12.8  
   Git commit:        6a30dfc  
   Built:             Thu Aug 29 05:26:49 2019  
   OS/Arch:           darwin/amd64  
   Experimental:      false  
  Server: Docker Engine - Community  
   Engine:  
    Version:          19.03.2  
    API version:      1.40 (minimum version 1.12)  
    Go version:       go1.12.8  
    Git commit:       6a30dfc  
    Built:            Thu Aug 29 05:32:21 2019  
    OS/Arch:          linux/amd64  
    Experimental:     false  
   containerd:  
    Version:          v1.2.6  
    GitCommit:        894b81a4b802e4eb2a91d1ce216b8817763c29fb  
   runc:  
    Version:          1.0.0-rc8  
    GitCommit:        425e105d5a03fabd737a126ad93d62a9eeede87f  
   docker-init:  
    Version:          0.18.0  
    GitCommit:        fec3683

When I called `mlflow.sagemaker.deploy()` with proper parameters, I saw the Endpoint creation failed in AWS SageMaker, suggesing me to check CloudWatch, and there was the same error message pasted above.

##### Deploy to SageMaker
To be added